In [1]:
import pandas as pd
import numpy as np

In [2]:
recipe = pd.read_csv(r"C:\Users\dheer\OneDrive - Binghamton University\Fall2021\SSIE510\data\recipes.csv")
reviews = pd.read_csv(r"C:\Users\dheer\OneDrive - Binghamton University\Fall2021\SSIE510\data\reviews.csv")

In [3]:
recipe.columns

Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [4]:
recipe["RecipeIngredientParts"]

0         c("blueberries", "granulated sugar", "vanilla ...
1         c("saffron", "milk", "hot green chili peppers"...
2         c("sugar", "lemons, rind of", "lemon, zest of"...
3         c("extra firm tofu", "eggplant", "zucchini", "...
4         c("plain tomato juice", "cabbage", "onion", "c...
                                ...                        
522512    c("fresh ginger", "unsalted butter", "dark bro...
522513    c("Dijon mustard", "garlic", "peppercorns", "s...
522514    c("half-and-half", "heavy cream", "brandy", "s...
522515    c("wasabi paste", "dill", "English cucumber", ...
522516    c("hard-boiled eggs", "breakfast sausage", "pa...
Name: RecipeIngredientParts, Length: 522517, dtype: object

In [5]:
recipe["RecipeInstructions"]

0         c("Toss 2 cups berries with sugar.", "Let stan...
1         c("Soak saffron in warm milk for 5 minutes and...
2         c("Into a 1 quart Jar with tight fitting lid, ...
3         c("Drain the tofu, carefully squeezing out exc...
4         c("Mix everything together and bring to a boil...
                                ...                        
522512    c("Preheat oven to 350&deg;F Grease an 8x8 cak...
522513    c("Position rack in center of oven and preheat...
522514    c("heat half and half and heavy cream to a sim...
522515    c("In a small bowl, combine mayo and wasabi pa...
522516    c("Mix sausage, panko, egg yolk and Wocestersh...
Name: RecipeInstructions, Length: 522517, dtype: object

In [6]:
recipe.drop(columns=["CookTime","PrepTime","TotalTime","DatePublished","AggregatedRating","ReviewCount","RecipeServings","RecipeYield","Images","RecipeYield","RecipeIngredientQuantities"],inplace=True)
recipe = recipe[recipe['Keywords'].notna()]
recipe.dropna(inplace=True)
recipe.drop_duplicates(inplace=True)

In [7]:
AverageRatings = pd.DataFrame(reviews.groupby("RecipeId")["Rating"].mean()).reset_index()

In [8]:
Reviewscounts = pd.DataFrame(reviews["RecipeId"].value_counts()).reset_index().rename(columns = {"RecipeId":"Number of Reviews","index":"RecipeId"})

In [9]:
masterdata = recipe.merge(Reviewscounts,on="RecipeId",how="left")
masterdata = masterdata.merge(AverageRatings,on="RecipeId",how="left")

In [10]:
masterdata = masterdata.sample(frac=0.6,random_state=42)

In [11]:
masterdata["RecipeInstructions"]

445886    "Mix all together but the chips, refrigerate a...
485163    c("Cake:  Preheat the oven to 350°F  Line the ...
394172    c("- Slice the onion (roughly 5mm width).", "-...
375427    c("Preheat oven to 375 degrees. Cook spaghetti...
300783    c("In a 9x13 casserole layer the sausage, mush...
                                ...                        
225865    c("In a large bowl,dissolve gelatin in boiling...
317118    c("Whisk all ingredients utnil well combined."...
197159    c("Preheat oven to 375 degrees.", "Cut, peel, ...
286257    c("Heat oven to 375. Grease or line with foil ...
111043    c("Bring rhubarb and jelly to a boil over medi...
Name: RecipeInstructions, Length: 303165, dtype: object

In [12]:
masterdata["Keywords"] = np.where(masterdata["Keywords"].str.contains("c()"),masterdata["Keywords"].str[2:-1],masterdata["Keywords"])
masterdata["RecipeIngredientParts"] = np.where(masterdata["RecipeIngredientParts"].str.contains("c()"),masterdata["RecipeIngredientParts"].str[2:-1],masterdata["RecipeIngredientParts"])
masterdata["RecipeInstructions"] = np.where(masterdata["RecipeInstructions"].str.startswith("c("),masterdata["RecipeInstructions"].str[2:-1],masterdata["RecipeInstructions"])

C:\Users\dheer\AppData\Local\Temp/ipykernel_41924/2844290131.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  masterdata["Keywords"] = np.where(masterdata["Keywords"].str.contains("c()"),masterdata["Keywords"].str[2:-1],masterdata["Keywords"])
C:\Users\dheer\AppData\Local\Temp/ipykernel_41924/2844290131.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  masterdata["RecipeIngredientParts"] = np.where(masterdata["RecipeIngredientParts"].str.contains("c()"),masterdata["RecipeIngredientParts"].str[2:-1],masterdata["RecipeIngredientParts"])


In [13]:
masterdata["RecipeInstructions"]

445886    "Mix all together but the chips, refrigerate a...
485163    "Cake:  Preheat the oven to 350°F  Line the bo...
394172    "- Slice the onion (roughly 5mm width).", "- P...
375427    "Preheat oven to 375 degrees. Cook spaghetti u...
300783    "In a 9x13 casserole layer the sausage, mushro...
                                ...                        
225865    "In a large bowl,dissolve gelatin in boiling w...
317118    "Whisk all ingredients utnil well combined.", ...
197159    "Preheat oven to 375 degrees.", "Cut, peel, an...
286257    "Heat oven to 375. Grease or line with foil an...
111043    "Bring rhubarb and jelly to a boil over medium...
Name: RecipeInstructions, Length: 303165, dtype: object

In [14]:
masterdata.drop(columns=["RecipeId","AuthorId","AuthorName"],inplace=True)
masterdata["Name"] = masterdata["Name"].str.replace("&amp;","and")
masterdata["Name"] = masterdata["Name"].str.replace("&amp","and")
masterdata["Description"] = masterdata["Description"].str.replace("&amp;","and")
masterdata["Description"] = masterdata["Description"].str.replace("&amp","and")

#masterdata.to_csv(r"C:\Users\dheer\OneDrive - Binghamton University\Fall2021\SSIE510\data\masterdata.csv",index=False)

In [30]:
list(masterdata.loc[masterdata["Name"] == "Crispy Chews"]["RecipeInstructions"].values)[0].split('", "')

['"In a large saucepan, heat the rice syrup and almond butter until creamy.',
 'Stir in the chocolate chips until they melt.',
 'Remove from the heat and stir in the crispy rice until well incorporated.',
 'Press into a shallow, square casserole dish.',
 'Allow to set until firm.',
 'Cut into squares and serve.',
 'Makes about 1 dozen."']

In [15]:
def flatten(t):
    return [item for sublist in t for item in sublist]
keywords = flatten(masterdata["Keywords"].str.split(",").to_list())
ingredients = flatten(masterdata["RecipeIngredientParts"].str.split(",").to_list())

In [16]:
keywords = [keyword.rstrip('"').lstrip(' "') for keyword in keywords]
ingredients = [ingredient.rstrip('"').lstrip(' "') for ingredient in ingredients]

In [17]:
len(keywords)

1517829

In [18]:
#tuple(set(keywords))

In [19]:
print(tuple(set(ingredients)))

('Italian Salad Dressing', 'buckwheat flakes', 'calrose rice', 'hot Italian sausage link', 'riesling wine', 'ground turmeric', 'fresh cumin seed', 'kumquats', 'nasturtium buds', 'yellow corn grits', 'Mock Feta Cheese ( Vegan Substitute )', 'green onions', 'istachio nut', 'stuffed green olive', 'Basic Pizza Dough', 'black cod steaks', 'powdered sugar substitute', 'Creme Fraiche', 'szechuan peppercorns', 'vegetarian Canadian bacon', 'ground fennel', 'tangerine juice concentrate', "Kittencal's Best Chicken Stock/Broth (Crock Pot Option)", 'yellow fin potatoes', 'Tony’s Pizza Dough', 'No Guilt Hot Fudge Sauce', 'baby portabella mushrooms', 'juice and zest of', 'cinnamon sticks', 'onion rings', "Sooz's Semi-Homemade  Salsa", 'graham cracker', 'Baking Mix Aka &quot;bisquick&quot;', 'tangerine peel', 'ground sumac', 'diced tomatoes with green pepper celery and onion', 'minced  fresh garlic cloves', 'Ginger Miso Paste', 'erman chocolate cake mix with pudding', 'unsweetened pineapple', 'fresh p

In [20]:
#tuple(masterdata["RecipeCategory"].unique())

In [23]:
getminmax =['Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent']
for nutri in getminmax:
    mini = masterdata[nutri].min()
    maxi = masterdata[nutri].max()
    print("{},min:{},max:{}".format(nutri,mini,maxi))

Calories,min:0.0,max:612854.6
FatContent,min:0.0,max:64368.1
SaturatedFatContent,min:0.0,max:26740.6
CholesterolContent,min:0.0,max:89892.0
SodiumContent,min:0.0,max:731056.4
CarbohydrateContent,min:0.0,max:108294.6
FiberContent,min:0.0,max:3012.0
SugarContent,min:0.0,max:90682.3
ProteinContent,min:0.0,max:18396.2


In [24]:
data = pd.read_csv("https://ssie510dss.s3.amazonaws.com/masterdata.csv")